# Scraping Review - Usajobs.gov

So it appears that the Usajobs.gov site has updated recently and because of their update, it's become much harder to scrape.  We hope that this underscores that 3rd party tools have sever limitations, and the need to learn good scraping fundamentals.  For the most part, scraping proficiency has a steap learning curve.  Even in the code of Usajobs.gov, there are many concepts standing in the way of our first few steps.

This material is meant to give you a jumpstart into project 4 so you can focus on the level of knowledge we've intended to present.  Perhaps you will absorb a bit more depth about this realm than you anticipated after reading through all of this.  From here, we will go over a few of the obstacles, and illustrate how to work around them.

_This case may seem overly complex but it's not every site but certainly it should give you a sense about what's waiting for you out in the wild.  Scraping is challenging yet rewarding!_

## First - Fetch Page with Requests

In [63]:
# Please only run this frame once to avoid hitting the site too hard all at once
import requests

# Search parameters -- for convenience 
search_term = "data"
location    = "San Francisco, California"

# Search and location get replaced in the next few lines
url = "https://www.usajobs.gov/Search/?Keyword=%s&Location=%s&homeRadPublic=public&search=Search&AutoCompleteSelected=False&CanSeekStatusJobs=False"
url = url % (search_term, location)

response = requests.get(url)
HTML = response.text  
HTML[0:500]           # view the first 500 characters of the HTML index document to see that it works

u'\r\n<!DOCTYPE html>\r\n<html lang="en" class="scrollbar">\r\n<head><meta charset="utf-8" /><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" /><title>\r\n\t\r\n    \r\n    USAJOBS - The Federal Government&rsquo;s Official Jobs Site\r\n\r\n\r\n</title>      \r\n       <link href="/Content/Styles/usaj-responsive?v=Wlro1xQyoRuae0CQQXV_1ql5IlgERwvWO5gPIEmCw701" rel="stylesheet"/>\r\n\r\n       <link href="/Content/theme-usaj/bundle?v=erLJ6YjgxUdod610aoJBdWnO1M-mnUwaJcPhhPrBN7k1" rel="stylesheet"/>\r\n\r\n\r\n\t\r\n\t\t<scr'

## First, how many results are there?
We should figure out how many pages should be scraped.  It looks like on our page, it's displaying 25 titles per page.

![](https://snag.gy/yIEZwD.jpg)

A few considerations:

- Check that you have at least 25 job titles
  - If you don't, no need to attempt to fetch the next page
- The number of pages you have to fetch is [total number results per page] / [total results]

We need to know the following in order to do sequential crawls of results per page:
- Are there more than one pages to scrape / crawl?
- How many pages are there?

In [67]:
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

# This xpath query will find the number of total results
results = Selector(text=HTML).xpath("//span[@class='pageset pager-totalRecords']/text()")

total_results = int(results[0].extract())
total_results

55

In [68]:
# This will basically scrape the page of job titles -- simply a test / example

results = Selector(text=HTML).xpath("//a[@class='jobTitleLink']")
# results.extract()
job_titles = []
for jobs in results:
    job_title = jobs.xpath("text()").extract()[0]
    job_titles.append(job_title)
    
job_titles

[u'Interior Designer',
 u'Construction Analyst (REAC)',
 u'IT Specialist (NETWORK)',
 u'Mobile Communications Operations Vehicle Operator',
 u'Management and Program Analyst, GS-0343-7/9 (Pathways Recent Graduates Program)',
 u'Portfolio Management Specialist',
 u'Portfolio Management Specialist',
 u'Network Intrusion Forensic Analyst',
 u'Geospatial Information System Specialist',
 u'Paralegal Specialist',
 u'Auditor',
 u'IT Auditor',
 u'Program Analyst',
 u'Chief Scientific and Technical Advisor for Aircraft Computer Software',
 u'IT Specialist',
 u'Auditor (Recent Graduate)',
 u'INTERNAL REVENUE AGENT',
 u'IT Specialist (PLCYPLN)',
 u'Construction Analyst (REAC)',
 u'Advanced Medical Support Assistant',
 u'Business Manager ((Category Management))',
 u'Landscape Architect',
 u'Human Resources Specialist',
 u'Computer Engineer - Applications Engineer (DHA)',
 u'Transportation Security Inspector - Explosives Detection Canine Handler']

## But why doesn't import.io scrape this!?

It could be that our target, usajobs.gov, has updated the way their site is navigated via javascript, rather than static html.  Import.io isn't super useful in this case.  We can use our xpath, requests, and Python smarts to find the goods once we can figure out how each search page is being rendered.

Inspecting the "network" traffic from Chrome's developer console, we can find that pages are fetched behind the scenes using asynchornous requests via Javascript:

![](https://snag.gy/Kdn0yo.jpg)

What does that mean!?!?  Well, when we click on a page number in our search results, javascript fetches the page from the server without reloading the current frame, then updates the search results with new content.  Because of this, import.io is largely confused.  By inspecting the request from the console, we can see that "XHR" requests from the "network" group in the console, says this is happening.

### XHR / Ajax Requests

Whenver you click on a page from the search results, Javascript makes a request to the usajobs.gov site and requests the next page.  From the "network" console of chrome, we can see the url is this:

https://www.usajobs.gov/Search/GetPageResults/?page=1&statusFilter=public&X-Requested-With=XMLHttpRequest

For subsequent pages, these are the pages it fetches:
- https://www.usajobs.gov/Search/GetPageResults/?page=2&statusFilter=public&X-Requested-With=XMLHttpRequest
- https://www.usajobs.gov/Search/GetPageResults/?page=3&statusFilter=public&X-Requested-With=XMLHttpRequest

**Notice the variable that is passed via URL "page=n"**

If we know the total number of results, then we can project the number of pages we can fetch in a row to automate our scraping process.

### But there's actually a little more to the story

So you might ask "well how does the URL above know about my "search term" or "location"?  The short answer is, it doesn't.  Upon further inspection of the usajobs.gov protocols, they are using cookies to link the above queries, to the originating query with the search terms and location related to it.  It's fairly easy to link these. 

[Check this video out for a further explanation about cookies.](https://www.youtube.com/watch?v=I01XMRo2ESg)

Actually, this "control" is a fairly unorthadox use of cookies and Ajax requests via web development.  From looking how usajobs.gov is authored, its unlikely to be a sophiticated control to prevent scraping and more to do with a lack of good development standards.




## Calculate number of pages

Onward, we can do a basic calculation to project the number of pages we can scrape.

In [69]:
number_of_pages = total_results / 25 # 25 = displayed per page
number_of_pages

2

Then we can fetch every single page, scraping it as we go along, adding the the job_titles[] list.  Notice in the **requests.get()** call, we are passing **cookies=response.cookies**.  This is necessary for the query to run or else it will return **400 bad request** because it expects the cookies from the original search query to be present so it knows what these requests are realted to (this might be considered bad practice by some web developers).  These subsequent urls are easy to generate by just swaping out the **page=n** variable in iteration.

In [73]:
job_titles = []

for page_number in range(1, number_of_pages + 2): # the government site can actually display more than the number of pages it thinks it should have
    
    next_page_url = "https://www.usajobs.gov/Search/GetPageResults/?page=%d&statusFilter=public&X-Requested-With=XMLHttpRequest" % page_number
    response = requests.get(url, cookies=response.cookies)
    HTML = response.text  
    
    # fetch page, map results, append to list
    print "Scraping page %d..." % page_number
    
    results = Selector(text=HTML).xpath("//a[@class='jobTitleLink']")    
    for jobs in results:
        job_titles.append(jobs.xpath("text()").extract()[0])
    

print "Completed scraping.. %d job titles scraped" % len(job_titles)



Scraping page 1...
Scraping page 2...
Scraping page 3...
Completed scraping.. 75 job titles scraped


In [60]:
# Let's check out the first 10 jobs scraped
print job_titles[0:10]
print 
print len(job_titles), "total jobs scraped"

[u'Interior Designer', u'Construction Analyst (REAC)', u'IT Specialist (NETWORK)', u'Mobile Communications Operations Vehicle Operator', u'Management and Program Analyst, GS-0343-7/9 (Pathways Recent Graduates Program)', u'Portfolio Management Specialist', u'Portfolio Management Specialist', u'Network Intrusion Forensic Analyst', u'Geospatial Information System Specialist', u'Paralegal Specialist']

75 total jobs scraped


## Conclusion

We can't do all the work for you, but since this project was intended to be easy to scrape with minimal programming, and it turns out the site has updated to make it just a little more annoying to scrape, this supplemental material should help you fill the gap of starting.  What's left for you to do is:

- Map more xpath queries to elements that you want
- Figure out the variables that control public vs state jobs and update the search query to differentiate between them

It's highly recommended that you try to modularize your code and make a function that scrapes each page.  Share your progress with each other and try to help each other out as much as possible.